Hier entwickeln wir das Modell (bzw. testen es aus) -> bevor man es in die richtige Codebase macht

notwendige Package

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import transformers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score as accuracy
#from sklearn.metrics import accuracy
#import tensorflow_text as text

2022-11-05 18:44:12.798271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Preprocessing -> TODO multiclass prediction

In [31]:
modelname = "prajjwal1/bert-tiny"
dataset = "twitter_sentiment_data - Davide.csv"

In [83]:
df = pd.read_csv("data/twitter_sentiment_data - Davide.csv")
df.loc[df.sentiment == -1, "sentiment"] = 2
df = df[df.sentiment < 2]
x_train,x_test,y_train,y_test = train_test_split(df.message.values,df.sentiment.values,test_size= 0.33)

Modelle -> Ergänzungen hier drunter einfach als tokenizer_modelname bzw. model_modelname

In [19]:
tokenizer = transformers.AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = transformers.TFAutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tiny bert hat bei mir 7 min 20 (bis zu 12:30) gebrauch (für 2/3 des gesamten Kaggle Datensatzes)

In [20]:
encoded_x_train = tokenizer(list(x_train),padding=True, return_tensors="np")
model.compile()
model.fit(encoded_x_train.data, y_train)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


643/643 [==============================] - 552s 849ms/step - loss: 0.5097


Prediction

In [21]:
encoded_x_test = tokenizer(list(x_test),padding=True, return_tensors="np")
y_pred = model.predict(encoded_x_test.data)
pred_labels = np.argmax(y_pred.logits, axis=1)
cert = np.max(y_pred.logits, axis=1)

317/317 [==============================] - 109s 340ms/step


Evaluation 

None


In [32]:
model.save_pretrained(f"models/{(modelname)}_{dataset}")

In [49]:
import os
os.path.exists("models/prajjwal1/bert-tiny_twitter_sentiment_data - Davide.csv/")

True

In [47]:
#model = transformers.TFPreTrainedModel.from_pretrained(f"models/{(modelname)}_{dataset}")
model = transformers.TFAutoModelForSequenceClassification.from_pretrained("models/prajjwal1/bert-tiny_twitter_sentiment_data - Davide.csv/")
model.compile()

Some layers from the model checkpoint at models/prajjwal1/bert-tiny_twitter_sentiment_data - Davide.csv/ were not used when initializing TFBertForSequenceClassification: ['dropout_22']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at models/prajjwal1/bert-tiny_twitter_sentiment_data - Davide.csv/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without f